In [8]:
import torch
import os
import pandas as pd
import warnings
import time  # Import the time module

from gluonts.dataset.pandas import PandasDataset
from gluonts.torch.model.patch_tst import PatchTSTLightningModule

In [2]:
# Load and preprocess data
file_path = "/Users/charlesmiller/Documents/Code/gluonts/model_builds/datasets/full.csv"
df = pd.read_csv(file_path, index_col=0, parse_dates=True)
df['v'] = df.index
df.reset_index(inplace=True, drop=True)
df = df.loc[df['symbol'].isin(['AAPL','SPY','QQQ','AMD','NVDA'])]
print(len(df))
df['time_idx'] = df.groupby('symbol').cumcount()
feature_columns = ['o','v','c','h','l','vw']
feature_df = df[['o','v','c','h','l','vw','time_idx','symbol']]
feature_df[feature_columns] = feature_df[feature_columns].astype('float32')

/var/folders/tq/_9q_lv1s2bndbr3tzzn08x4r0000gn/T/ipykernel_17117/1973061198.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_path, index_col=0, parse_dates=True)


504200


/var/folders/tq/_9q_lv1s2bndbr3tzzn08x4r0000gn/T/ipykernel_17117/1973061198.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df[feature_columns] = feature_df[feature_columns].astype('float32')


In [3]:
# Split data into training, validation, and test sets
train_df = feature_df[feature_df['time_idx'] < 80000]
validation_df = feature_df.loc[(feature_df['time_idx'] >= 80000) & (feature_df['time_idx'] < 90000)]
test_df = feature_df[feature_df['time_idx'] >= 90000]

print(f"Train shape: {train_df.shape}")
print(f"Train Symbols: {train_df['symbol'].unique()}")
print(f"Validation shape: {validation_df.shape}")
print(f"Validation Symbols: {validation_df['symbol'].unique()}")
print(f"Test shape: {test_df.shape}")
print(f"Test Symbols: {test_df['symbol'].unique()}")

# Create GluonTS datasets
training_data = PandasDataset.from_long_dataframe(
    train_df, timestamp="time_idx", target="c", item_id="symbol")
validation_data = PandasDataset.from_long_dataframe(
    validation_df, timestamp="time_idx", target="c", item_id="symbol")
testing_data = PandasDataset.from_long_dataframe(
    test_df, timestamp="time_idx", target="c", item_id="symbol")

Train shape: (400000, 8)
Train Symbols: ['AAPL' 'AMD' 'NVDA' 'QQQ' 'SPY']
Validation shape: (50000, 8)
Validation Symbols: ['SPY' 'QQQ' 'AAPL' 'AMD' 'NVDA']
Test shape: (54200, 8)
Test Symbols: ['SPY' 'QQQ' 'AAPL' 'AMD' 'NVDA']


In [10]:
best_model_path = "/Users/charlesmiller/Documents/Code/gluonts/lightning_logs/version_38/checkpoints/epoch=66-step=34304.ckpt"
model = PatchTSTLightningModule.load_from_checkpoint(best_model_path)
# best_estimator = PatchTSTEstimator.load_from_checkpoint(best_model_path)

In [ ]:
features = test_df[feature_columns].values
features = torch.tensor(features, dtype=torch.float32)
labels = test_df['c'].values
labels = torch.tensor(labels, dtype=torch.float32)

In [16]:
model.eval()
model.to('mps')
predictions = model(features, labels)

TypeError: forward() missing 1 required positional argument: 'past_observed_values'